In [1]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")
import time

In [2]:
#date_range

In [3]:
workouts = pd.read_csv('workouts.csv')

In [4]:
def epoch_end_convert(df):
    '''
    takes dataframe and converts date column to epoch milliseconds
    inputs:
        df = dataframe name
    '''
    try:
        return (int(time.mktime(time.strptime(df['end'], '%Y-%m-%d %H:%M:%S'))) - 14400)*1000
    except:
        #handles activities that have fractions of a second included
        return (int(time.mktime(time.strptime(df['end'][:19], '%Y-%m-%d %H:%M:%S'))) - 14400)*1000

In [5]:
#workouts_subset['end_epoch'] = workouts_subset.apply(epoch_end_convert, axis=1)
workouts['end_epoch'] = workouts.apply(epoch_end_convert, axis=1)

In [6]:
workouts.head()

,user_id,start,end,sport_id,name,intensity_score,z1,z2,z3,z4,z5,end_epoch
0,828,2015-09-15 20:08:00,2015-09-15 22:56:00,35,Track & Field,20.508564,1066,1810,1432,4668,679,1442357760000
1,828,2015-09-19 12:46:00,2015-09-19 14:09:00,35,Track & Field,16.823285,506,601,1861,696,22,1442671740000
2,828,2015-09-20 13:15:00,2015-09-20 16:00:00,35,Track & Field,20.506562,929,1302,1329,5353,321,1442764800000
3,828,2015-09-21 18:55:34.582,2015-09-21 18:58:37.876,1,Cycling,4.278678,8,170,5,0,0,1442861917000
4,828,2015-09-21 19:19:00,2015-09-21 21:12:00,35,Track & Field,19.176787,655,1757,1372,2781,28,1442869920000


#Adding in Time Periods (For Either Weeks or Races)

In [48]:
date_markers = pd.read_csv('epoch_dates.csv')

In [49]:
date_markers.head()

,ET_Date,date_start_epoch,race_period,bike_period,race_period_start,race_period_end,bike_period_start,bike_period_end
0,9/15/2015 0:00,1442289600000,1,1,1,0,1,0
1,9/16/2015 0:00,1442376000000,1,1,0,0,0,0
2,9/17/2015 0:00,1442462400000,1,1,0,0,0,0
3,9/18/2015 0:00,1442548800000,1,1,0,0,0,0
4,9/19/2015 0:00,1442635200000,1,1,0,0,0,0


In [39]:
#date_markers = date_markers[date_markers['race_period_start'] > 0].reset_index()
#dictionary of race period to start epoch
#race_start_dict = {}
#for i in range(0, len(date_markers)):
#    race_start_dict[date_markers.values[i][3]] = date_markers.values[i][2]

#date_markers = date_markers[date_markers['bike_period_start'] > 0].reset_index()
#bike_start_dict = {}
#for i in range(0, len(date_markers)):
#    bike_start_dict[date_markers.values[i][4]] = date_markers.values[i][2]


In [40]:
race_start_dict

{1L: 1442289600000L,
 2L: 1443240000000L,
 3L: 1445054400000L,
 4L: 1446264000000L,
 5L: 1447473600000L}

In [41]:
bike_start_dict

{1L: 1442289600000L,
 2L: 1443499200000L,
 3L: 1444104000000L,
 4L: 1444708800000L,
 5L: 1445313600000L,
 6L: 1446004800000L,
 7L: 1446609600000L,
 8L: 1447214400000L,
 9L: 1447819200000L}

In [67]:
def add_race(df):
    if (df.end_epoch >= race_start_dict[1]) and (df.end_epoch < race_start_dict[2]):
        return 1
    elif (df.end_epoch >= race_start_dict[2]) and (df.end_epoch < race_start_dict[3]):
        return 2
    elif (df.end_epoch >= race_start_dict[3]) and (df.end_epoch < race_start_dict[4]):
        return 3
    elif (df.end_epoch >= race_start_dict[4]) and (df.end_epoch < race_start_dict[5]):
        return 4
    elif (df.end_epoch >= race_start_dict[5]) and (df.end_epoch < 1448164800000):
        return 5
    else:
        return 0


In [68]:
def add_bike(df):
    if (df.end_epoch >= bike_start_dict[1]) and (df.end_epoch < bike_start_dict[2]):
        return 1
    elif (df.end_epoch >= bike_start_dict[2]) and (df.end_epoch < bike_start_dict[3]):
        return 2
    elif (df.end_epoch >= race_start_dict[3]) and (df.end_epoch < bike_start_dict[4]):
        return 3
    elif (df.end_epoch >= bike_start_dict[4]) and (df.end_epoch < bike_start_dict[5]):
        return 4
    elif (df.end_epoch >= bike_start_dict[5]) and (df.end_epoch < bike_start_dict[6]):
        return 5
    elif (df.end_epoch >= bike_start_dict[6]) and (df.end_epoch < bike_start_dict[7]):
        return 6
    elif (df.end_epoch >= bike_start_dict[7]) and (df.end_epoch < bike_start_dict[8]):
        return 7
    elif (df.end_epoch >= bike_start_dict[8]) and (df.end_epoch < bike_start_dict[9]):
        return 8
    elif (df.end_epoch >= bike_start_dict[9]):
        return 9

In [73]:
def add_periods(date_markers, df, buildup_days=0, version=None):
    '''
    Adds race periods to the input dataframe 'df'
        Inputs: 
            date_markers: dataframe with the ET date, the start epoch time of that date, and which race period the date is in
            df: the dataframe to add these date markers to
            buildup_days: if 0, we use the entire time period before the race/bike, after the previous one
                if not 0, and instead a value "n", we look at the "n" days before the race/bike as the "period"
            version: takes two values: "bike", or "race", and treats appropriately
    '''
    if version==None:
        print "Enter Either 'race' or 'bike'!"
    if buildup_days==0:
        if version=="race":
            #Collapse the data frame into only the start and end dates for each race period
            date_markers = date_markers[date_markers['race_period_start'] > 0].reset_index()
            
            #dictionary of race period to start epoch
            race_start_dict = {}
            for i in range(0, len(date_markers)):
                race_start_dict[date_markers.values[i][3]] = date_markers.values[i][2]
            
            #Now label each race period as 1, 2, 3, 4, or 5
            df['race_period'] = df.apply(add_race, axis=1)
            
        elif version=="bike":
            #same process, but using the bike start dates
            date_markers = date_markers[date_markers['bike_period_start'] > 0].reset_index()
            bike_start_dict = {}
            for i in range(0, len(date_markers)):
                bike_start_dict[date_markers.values[i][4]] = date_markers.values[i][2]
            df['bike_period'] = df.apply(add_bike, axis=1)
        return df

In [74]:
temp = add_periods(date_markers=date_markers, df=workouts, buildup_days=0, version="race")

In [87]:
group = temp.sort(['user_id', 'race_period', 'end_epoch'], ascending=[True, True, False])

In [90]:
group

,user_id,start,end,sport_id,name,intensity_score,z1,z2,z3,z4,z5,end_epoch,race_period
8,828,2015-09-25 18:30:00,2015-09-25 20:47:00,0,Running,19.797324,1453,1601,1803,2406,630,1443214020000,1
7,828,2015-09-23 20:47:48.968,2015-09-23 20:58:34.201,44,Yoga,4.081088,142,13,0,0,0,1443041914000,1
6,828,2015-09-23 19:31:48.525,2015-09-23 20:21:16.705,35,Track & Field,16.790126,126,499,703,1452,89,1443039676000,1
5,828,2015-09-22 20:31:00,2015-09-22 22:00:00,35,Track & Field,18.825382,767,992,1291,806,1323,1442959200000,1
4,828,2015-09-21 19:19:00,2015-09-21 21:12:00,35,Track & Field,19.176787,655,1757,1372,2781,28,1442869920000,1
3,828,2015-09-21 18:55:34.582,2015-09-21 18:58:37.876,1,Cycling,4.278678,8,170,5,0,0,1442861917000,1
2,828,2015-09-20 13:15:00,2015-09-20 16:00:00,35,Track & Field,20.506562,929,1302,1329,5353,321,1442764800000,1
1,828,2015-09-19 12:46:00,2015-09-19 14:09:00,35,Track & Field,16.823285,506,601,1861,696,22,1442671740000,1
0,828,2015-09-15 20:08:00,2015-09-15 22:56:00,35,Track & Field,20.508564,1066,1810,1432,4668,679,1442357760000,1
30,828,2015-10-16 17:55:00,2015-10-16 20:16:00,35,Track & Field,19.064909,2052,1557,1584,1663,476,1445026560000,2


,user_id,start,end,sport_id,name,intensity_score,z1,z2,z3,z4,z5,end_epoch,race_period,count
8,828,2015-09-25 18:30:00,2015-09-25 20:47:00,0,Running,19.797324,1453,1601,1803,2406,630,1443214020000,1,13
7,828,2015-09-23 20:47:48.968,2015-09-23 20:58:34.201,44,Yoga,4.081088,142,13,0,0,0,1443041914000,1,13
6,828,2015-09-23 19:31:48.525,2015-09-23 20:21:16.705,35,Track & Field,16.790126,126,499,703,1452,89,1443039676000,1,13
5,828,2015-09-22 20:31:00,2015-09-22 22:00:00,35,Track & Field,18.825382,767,992,1291,806,1323,1442959200000,1,13
4,828,2015-09-21 19:19:00,2015-09-21 21:12:00,35,Track & Field,19.176787,655,1757,1372,2781,28,1442869920000,1,13
3,828,2015-09-21 18:55:34.582,2015-09-21 18:58:37.876,1,Cycling,4.278678,8,170,5,0,0,1442861917000,1,13
2,828,2015-09-20 13:15:00,2015-09-20 16:00:00,35,Track & Field,20.506562,929,1302,1329,5353,321,1442764800000,1,13
1,828,2015-09-19 12:46:00,2015-09-19 14:09:00,35,Track & Field,16.823285,506,601,1861,696,22,1442671740000,1,13
0,828,2015-09-15 20:08:00,2015-09-15 22:56:00,35,Track & Field,20.508564,1066,1810,1432,4668,679,1442357760000,1,13
30,828,2015-10-16 17:55:00,2015-10-16 20:16:00,35,Track & Field,19.064909,2052,1557,1584,1663,476,1445026560000,2,13
